<a href="https://colab.research.google.com/github/imabari/rakuten_tools/blob/main/rakuten_tower_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 設定に絞り込みたいeNB-LCIDを入力
2. 上部メニューより「ランタイム」を選択
3. 「すべてのセルを実行」を選択
4. アップロードで一時停止しているので参照ボタンを押してTowerCollectorのCSVファイルをアップロード
5. 最後にマップが表示される

In [ ]:
f = lambda enb, lcid: (enb << 8) | lcid

# 設定

In [ ]:
# eNB-LCID

cells = [f(344574, 1)]

# 複数の場合
# cells = [f(737430, 10), f(737430, 11), f(737430, 12)]

cells

# アップロード

参照ボタンを押してCSVファイルをアップロード

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# プログラム

In [ ]:
!pip install -U pandas

In [ ]:
import pandas as pd

In [ ]:
import folium
from folium.features import DivIcon

In [ ]:
def make_map(df, zoom=14):

    lat = df["lat"].mean()
    lon = df["lon"].mean()

    map = folium.Map(tiles = None, location=[lat, lon], zoom_start=zoom)

    # Googleマップ標準
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map",
        attr="<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>",
    ).add_to(map)

        # Googleマップ航空写真
    folium.raster_layers.TileLayer(
        "https://{s}.google.com/vt/lyrs=s,h&x={x}&y={y}&z={z}",
        subdomains=["mt0", "mt1", "mt2", "mt3"],
        name="Google Map(航空写真)",
        attr="<a href='https://developers.google.com/maps/documentation'>© Google</a>",
    ).add_to(map)

    fg1 = folium.FeatureGroup(name="Good").add_to(map)
    fg2 = folium.FeatureGroup(name="Bad").add_to(map)

    for i, r in df.iterrows():

        # レイヤーしきい値
        condition = r["rsrp"] > -80

        folium.CircleMarker(
            location=[r.lat, r.lon],
            radius=10,
            color=r.color,
            fill=True,
            tooltip=r["id"],
        ).add_to(fg1 if condition else fg2)

        folium.Marker(
            location=[r.lat, r.lon],
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="text-align:center; font-size: 12pt; font-weight: bold">{r["ta"]}</div>',
            ),
            tooltip=f'{r["id"]}<br />TA:{r["ta"]}<br />RSRP:{r["rsrp"]}',
        ).add_to(fg1 if condition else fg2)
    
    folium.LayerControl().add_to(map)

    return map

# データラングリング

In [ ]:
df0 = pd.read_csv(fn, parse_dates=["measured_at"], dtype={"ta": "Int64"}).dropna(how="all", axis=1)
df0

In [ ]:
df0.dtypes

In [ ]:
# バンド指定
df1 = df0.query("mcc == 440 & mnc == 11").copy()
df1

In [ ]:
# 日時変換
df1["measured_at"] = df1["measured_at"].dt.tz_convert("Asia/Tokyo").dt.tz_localize(None)

In [ ]:
df1["id"] = df1["short_cell_id"].astype(str) + "-" + df1["rnc"].astype(str)

In [ ]:
# 色分け（青 > 緑 > 黄 > 橙 > 赤）

df1["color"] = pd.cut(
    df1["rsrp"],
    [-140, -100, -90, -80, -40],
    labels=["red", "orange", "yellow", "green"],
).astype(str)

idx = df1.groupby("cell_id")["rsrp"].transform(max) == df1["rsrp"]
df1.loc[idx, "color"] = "blue"

# 地図

## 全部

In [ ]:
map1 = make_map(df1)
map1

In [ ]:
map1.save("map1.html")

## 絞り込み

In [ ]:
df2 = df1[df1["cell_id"].isin(cells)].copy()
df2

In [ ]:
map2 = make_map(df2, 16)
map2

In [ ]:
map2.save("map2.html")

# ダウンロード

In [ ]:
from google.colab import files

files.download("map2.html")